# Day Multi-Turn Integration Test

낮 파이프라인을 여러 턴에 걸쳐 자동으로 실행합니다.

```
DayController.process()   ← tool_call (LLM) → tool 실행 → rule_engine
  ↓
wsm.apply_delta()         ← 상태 업데이트
  ↓
EndingChecker.check()     ← 엔딩 체크
  ↓
NarrativeLayer.render()   ← 나레이션 생성 (LLM)
```

각 단계를 개별적으로 try/except해서 **어느 단계에서 연결이 끊어졌는지** 확인합니다.

In [1]:
# ── 0. 환경 설정 ────────────────────────────────────────────
import sys, logging, traceback
from pathlib import Path
from dotenv import load_dotenv

ROOT = Path.cwd()
while not (ROOT / "app").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

load_dotenv(ROOT / ".env")

# 로그: WARNING 이상만 출력 (INFO 억제), 에러는 명시적으로 출력
logging.basicConfig(level=logging.WARNING, format="%(name)s | %(levelname)s | %(message)s")

print(f"Project root : {ROOT}")

Project root : c:\Users\amate\GIT\game_demo


In [2]:
# ── 0-1. LoRA / PostProcess 로그 캡처 설정 ──────────────────────────────────
import logging


class LogCapture(logging.Handler):
    """특정 로거의 레코드를 버퍼에 캡처하는 핸들러"""

    def __init__(self):
        super().__init__(level=logging.DEBUG)
        self.records: list[logging.LogRecord] = []

    def emit(self, record: logging.LogRecord) -> None:
        self.records.append(record)

    def clear(self) -> None:
        self.records.clear()


_log_capture = LogCapture()

# 관심 로거에 핸들러 부착 (DEBUG 레벨 활성화, root 전파 차단)
_CAPTURE_LOGGERS = ["app.tools", "app.llm.engine", "app.postprocess"]
for _name in _CAPTURE_LOGGERS:
    _lg = logging.getLogger(_name)
    _lg.setLevel(logging.DEBUG)
    _lg.propagate = False   # root logger로 전파 차단 (이중 출력 방지)
    if _log_capture not in _lg.handlers:
        _lg.addHandler(_log_capture)

print("LogCapture 설정 완료:", _CAPTURE_LOGGERS)

LogCapture 설정 완료: ['app.tools', 'app.llm.engine', 'app.postprocess']


In [3]:
# ── 1. 시나리오 로드 & WorldState 초기화 ────────────────────
from app.loader import ScenarioLoader
from app.state import get_world_state_manager

SCENARIO_ID = "coraline"
USER_ID     = "test_runner"

loader = ScenarioLoader(ROOT / "scenarios")
assets = loader.load(SCENARIO_ID)

wsm   = get_world_state_manager()
wsm.reset_state(USER_ID, SCENARIO_ID)            # 항상 깨끗하게 시작
world = wsm.get_state(USER_ID, SCENARIO_ID, assets)

print(f"시나리오  : {assets.scenario.get('title')}")
print(f"NPC       : {list(world.npcs.keys())}")
print(f"초기 vars : {world.vars}")
print(f"초기 인벤 : {world.inventory}")
print(f"턴 제한   : {assets.get_turn_limit()}")

시나리오  : 인형의 집 (Puppet Home)
NPC       : ['stepmother', 'stepfather', 'brother', 'grandmother', 'dog_baron']
초기 vars : {'humanity': 100, 'suspicion_level': 0, 'day': 1, 'house_on_fire': False}
초기 인벤 : ['warm_black_tea']
턴 제한   : 50


In [4]:
# ── 2. 컴포넌트 임포트 ──────────────────────────────────────
from app.day_controller import get_day_controller
from app.ending_checker import EndingChecker
from app.narrative import get_narrative_layer
from app.lock_manager import get_lock_manager
from app.status_effect_manager import get_status_effect_manager
from app.item_acquirer import get_item_acquirer

controller      = get_day_controller()
ending_checker  = EndingChecker()
narrative       = get_narrative_layer()
lock_manager    = get_lock_manager()
sem             = get_status_effect_manager()
acquirer        = get_item_acquirer()

print("컴포넌트 초기화 완료")

컴포넌트 초기화 완료


In [5]:
# ── 4. 단계별 헬퍼 ──────────────────────────────────────────
OK   = "✅"
FAIL = "❌"

def status(label: str, ok: bool, detail: str = "") -> str:
    icon = OK if ok else FAIL
    line = f"  {icon} {label}"
    if detail:
        line += f" | {detail}"
    return line


def _print_llm_logs() -> None:
    """캡처된 로그에서 핵심 정보만 추출해 출력 후 버퍼 클리어"""
    records = list(_log_capture.records)
    _log_capture.clear()

    lora_used    = None
    token_counts = []   # (label, tokens)

    for rec in records:
        msg  = rec.getMessage()
        name = rec.name

        if name == "app.llm.engine":
            # LoRA 어댑터 사용 여부
            if "LoRA 어댑터 사용" in msg:
                lora_used = msg.split("LoRA 어댑터 사용")[-1].strip()
            # vLLM 응답 토큰 수
            elif "[vLLM Response]" in msg and "tokens=" in msg:
                try:
                    tokens = int(msg.split("tokens=")[-1].strip())
                    model_part = msg.split("|")[0].replace("[vLLM Response]", "").strip()
                    token_counts.append((model_part, tokens))
                except ValueError:
                    pass
            # LoRA 출력 원문
            elif "[LoRA 출력]" in msg:
                print(f"      📝 {msg}")

        elif name == "app.tools" and "[LoRA raw]" in msg:
            raw = msg.split("[LoRA raw]", 1)[-1].strip()
            print(f"      📝 LoRA raw    | {raw}")

        elif name == "app.postprocess":
            print(f"      🔄 PostProcess | {msg}")

    # 요약 출력
    if lora_used:
        print(f"      🔧 LoRA 어댑터  | {lora_used}")
    for label, tokens in token_counts:
        print(f"      🤖 LLM 호출    | {tokens} tokens")


def run_turn(user_input: str, world_state, turn_no: int):
    """
    한 턴을 실행하고 각 단계의 성공/실패를 출력합니다.
    game.py process_turn 파이프라인과 동일:
      LockManager → StatusEffectManager → DayController
      → Delta 적용 → ItemAcquirer → day_action_log
      → EndingChecker → NarrativeLayer

    Returns: (updated_world_state, ended: bool)
    """
    print(f"\n{'═' * 60}")
    print(f"  TURN {turn_no}  |  {user_input}")
    print(f"{'═' * 60}")

    tool_result  = None
    world_after  = world_state
    ended        = False

    _log_capture.clear()  # 이전 로그 초기화

    # ── Step 1: LockManager - 정보 해금 ──────────────────────
    try:
        locks_data = assets.extras.get("locks", {})
        lock_result = lock_manager.check_unlocks(world_state, locks_data)
        newly = lock_result.newly_unlocked
        detail = f"새 해금={newly}" if newly else "변화 없음"
        print(status("LockManager", True, detail))
    except Exception as e:
        print(status("LockManager", False, str(e)))
        traceback.print_exc()

    # ── Step 1.5: StatusEffectManager - 만료 효과 해제 ───────
    try:
        sem.tick(world_state.turn, world_state)
        print(status("StatusEffectManager", True, f"turn={world_state.turn}"))
    except Exception as e:
        print(status("StatusEffectManager", False, str(e)))
        traceback.print_exc()

    # ── Step 2: DayController.process() ─────────────────────
    try:
        tool_result = controller.process(user_input, world_state, assets)
        log = controller.decision_log[-1]
        sel = log["tool_selection"]
        detail = f"tool={sel['tool_name']}  intent={log['intent']}  args={sel['args']}"
        print(status("DayController", True, detail))
        _print_llm_logs()   # ← 핵심 LLM 정보만 요약 출력
        if tool_result.npc_response:
            print(f"      NPC 응답: {tool_result.npc_response}")
        if tool_result.event_description:
            for ev in tool_result.event_description:
                print(f"      사건: {ev}")
    except Exception as e:
        print(status("DayController", False, str(e)))
        traceback.print_exc()
        _log_capture.clear()

    # ── Step 3: wsm.apply_delta() ───────────────────────────
    try:
        delta = tool_result.state_delta if tool_result else {}
        delta["turn_increment"] = delta.get("turn_increment", 1)
        wsm.persist(USER_ID, SCENARIO_ID, world_state)
        world_after = wsm.apply_delta(USER_ID, SCENARIO_ID, delta, assets)
        print(status("apply_delta", True,
                     f"turn {world_state.turn}→{world_after.turn}  "
                     f"vars={world_after.vars}"))
    except Exception as e:
        print(status("apply_delta", False, str(e)))
        traceback.print_exc()

    # ── Step 3.5: ItemAcquirer - 자동 아이템 획득 스캔 ────────
    try:
        acq_result = acquirer.scan(world_after, assets)
        if acq_result.newly_acquired:
            wsm.persist(USER_ID, SCENARIO_ID, world_after)
            world_after = wsm.apply_delta(USER_ID, SCENARIO_ID, acq_result.acquisition_delta, assets)
            for acq_item_id in acq_result.newly_acquired:
                acq_item_def = assets.get_item_by_id(acq_item_id)
                acq_item_name = acq_item_def.get("name", acq_item_id) if acq_item_def else acq_item_id
                if tool_result:
                    tool_result.event_description.append(f"'{acq_item_name}'을(를) 발견했다!")
                print(f"      아이템 획득: {acq_item_name}")
            print(status("ItemAcquirer", True, f"획득={acq_result.newly_acquired}"))
        # 획득 없으면 출력 생략 (노이즈 감소)
    except Exception as e:
        print(status("ItemAcquirer", False, str(e)))
        traceback.print_exc()

    # ── Step 4: EndingChecker.check() ───────────────────────
    try:
        ending_result = ending_checker.check(world_after, assets)
        if ending_result.reached:
            detail = f"🎬 {ending_result.ending.name}  ({ending_result.ending.ending_id})"
            ended = True
            print(status("EndingChecker", True, detail))
        # 미도달은 출력 생략 (노이즈 감소)
    except Exception as e:
        print(status("EndingChecker", False, str(e)))
        traceback.print_exc()
        ending_result = None

    # ── Step 5: NarrativeLayer.render() ─────────────────────
    try:
        if ended and ending_result:
            ending_info = {
                "ending_id":       ending_result.ending.ending_id,
                "name":            ending_result.ending.name,
                "epilogue_prompt": ending_result.ending.epilogue_prompt,
            }
            narrative_text = narrative.render_ending(ending_info, world_after, assets)
            mode = "ending"
        else:
            ev    = tool_result.event_description if tool_result else []
            delta = tool_result.state_delta if tool_result else {}
            npc_r = tool_result.npc_response if tool_result else None
            narrative_text = narrative.render(
                world_state=world_after,
                assets=assets,
                event_description=ev,
                state_delta=delta,
                npc_response=npc_r,
            )
            mode = "day"
        _print_llm_logs()   # ← 나레이션 LLM 토큰 정보
        print(status("NarrativeLayer", True, f"mode={mode}  len={len(narrative_text)}자"))
    except Exception as e:
        print(status("NarrativeLayer", False, str(e)))
        traceback.print_exc()
        narrative_text = "(나레이션 생성 실패)"

    # ── 나레이션 출력 ────────────────────────────────────────
    print(f"\n{'─' * 50}")
    print(narrative_text)
    print(f"{'─' * 50}")

    return world_after, ended

## 6. 인터랙티브 모드

직접 입력해서 대화를 진행합니다. (최대 10턴)

- 각 턴마다 위와 동일한 파이프라인 단계 리포트가 출력됩니다.
- 엔딩 도달 시 자동 종료됩니다.
- **종료**: `q` 또는 `quit` 입력

먼저 **6-1 셀**로 WorldState를 초기화한 뒤, **6-2 셀**을 실행하세요.

In [ ]:
# ── 6-1. 인터랙티브 WorldState 초기화 ───────────────────────
# (자동 테스트와 독립적으로 초기 상태에서 시작)
wsm.reset_state(USER_ID, SCENARIO_ID)
_iworld = wsm.get_state(USER_ID, SCENARIO_ID, assets)
_iturn  = 1

print(f"시나리오  : {assets.scenario.get('title')}")
print(f"NPC       : {list(_iworld.npcs.keys())}")
print(f"초기 vars : {_iworld.vars}")
print(f"턴 제한   : {assets.get_turn_limit()}")
print()
print("초기화 완료 — 아래 셀을 실행해 인터랙티브 모드 시작")

In [ ]:
# ── 6-2. 인터랙티브 루프 (최대 10턴) ───────────────────────
MAX_TURNS = 10

print("=" * 60)
print(f"  인터랙티브 모드  |  Turn {_iturn}  |  최대 {MAX_TURNS}턴")
print(f"  종료: q 또는 quit")
print("=" * 60)

while _iturn <= MAX_TURNS:
    # ── 입력 ─────────────────────────────────────────────────
    try:
        text = input(f"\n[Turn {_iturn}/{MAX_TURNS}] 입력 > ").strip()
    except (EOFError, KeyboardInterrupt):
        print("\n종료합니다.")
        break

    if not text:
        continue
    if text.lower() in ("q", "quit", "exit"):
        print("종료합니다.")
        break

    # ── 파이프라인 실행 ───────────────────────────────────────
    _iworld, game_over = run_turn(text, _iworld, _iturn)

    # ── 상태 요약 ─────────────────────────────────────────────
    print(f"\n  [현재 상태]")
    print(f"    Turn {_iturn} → {_iworld.turn}  |  vars: {_iworld.vars}")
    for npc_id, npc_state in _iworld.npcs.items():
        npc_data = assets.get_npc_by_id(npc_id) or {}
        name = npc_data.get("name", npc_id)
        print(f"    [{name}] {npc_state.stats}")
    if _iworld.inventory:
        print(f"    inventory: {_iworld.inventory}")

    _iturn += 1

    if game_over:
        print(f"\n{'=' * 60}")
        print("  게임 종료 (엔딩 도달)")
        print(f"{'=' * 60}")
        break
    if _iturn > MAX_TURNS:
        print(f"\n{'=' * 60}")
        print(f"  {MAX_TURNS}턴 완료 — 루프 종료")
        print(f"{'=' * 60}")

  인터랙티브 모드  |  Turn 1  |  최대 10턴
  종료: q 또는 quit

════════════════════════════════════════════════════════════
  TURN 1  |  엄마 집에 가고 싶어
════════════════════════════════════════════════════════════
  ✅ LockManager.check_unlocks | result=newly_unlocked=[] all_unlocked_ids=set()
  ✅ StatusEffectManager.tick | turn=1


app.llm.engine | INFO | [LLM Init] vLLM 연결: base_url=https://nontheatrical-judiciarily-susanne.ngrok-free.dev/, base_model=Qwen/Qwen2.5-7B-Instruct
app.llm.engine | INFO | [LLM Init] backend=vLLM, model=Qwen/Qwen2.5-7B-Instruct
app.llm.engine | DEBUG | [LLM Generate] backend=vLLM, model=base
app.llm.engine | INFO | [vLLM Request] model=Qwen/Qwen2.5-7B-Instruct (base)
C:\Users\amate\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
app.llm.engine | INFO | vLLM 중국어 차단 토큰 수: 26123
app.llm.engine | INFO | [vLLM Response] model=Qwen/Qwen2.5-7B-Instruct (base) | tokens=244
app.llm.engine | DEBUG | [vLLM Output] - "주변을 살펴본다" → {"tool_name": "action", "args": {"action": "주변을 살펴본다"}, "intent": "investigate"}
- "가족사진을 바론에

  ✅ DayController.process | tool=action  intent=neutral  args={'action': '엄마 집으로 간다'}
      사건: 엄마 집으로 가서 새로운 환경을 경험
  ✅ wsm.apply_delta | turn 1→2  vars={'humanity': 100, 'suspicion_level': 0, 'day': 1, 'house_on_fire': False, 'status_effects': []}
  ✅ ItemAcquirer.scan | newly_acquired=[]
  ✅ day_action_log.append | log_count=1
  ✅ EndingChecker.check | 미도달


app.llm.engine | INFO | [vLLM Response] model=Qwen/Qwen2.5-7B-Instruct (base) | tokens=1
app.llm.engine | DEBUG | [vLLM Output]  


  ✅ NarrativeLayer.render | mode=day  len=10자

──────────────────────────────────────────────────
(서술 생성 실패)
──────────────────────────────────────────────────

  [현재 상태]
    Turn 1 → 2  |  vars: {'humanity': 100, 'suspicion_level': 0, 'day': 1, 'house_on_fire': False, 'status_effects': []}
    [새엄마 (엘리노어)] {'affection': 50, 'fear': 80, 'humanity': 0, 'plus_hits': 0, 'minus_hits': 0, 'critical_hits': 0}
    [새아빠 (아더)] {'affection': 30, 'fear': 60, 'humanity': 20}
    [동생 (루카스)] {'affection': 60, 'fear': 40, 'humanity': 50}
    [할머니 (마가렛)] {'affection': 20, 'fear': 10, 'humanity': 10}
    [강아지 (바론)] {'affection': 70, 'fear': 0, 'humanity': 100}
    inventory: ['warm_black_tea']
